### RAGを使って一問一答
1. Model読み込み
2. PromptTemplateの設定
3. FAISSのvector dataを取得
4. 類似ドキュメントの取得
5. 解答を得る

In [ ]:
import os
from langchain_community.vectorstores import FAISS
from langchain_openai import (
    AzureOpenAIEmbeddings,
    OpenAIEmbeddings,
    AzureChatOpenAI,
    ChatOpenAI
)
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
load_dotenv('../.env')

#### 1. Model読み込み

In [ ]:
# emmbeddingsのモデルを取得
embeddings = None
if os.getenv('AZURE_OPENAI_API_KEY') != "":
    # Azureの場合
    embeddings = AzureOpenAIEmbeddings(
        azure_deployment="embedding",
        openai_api_version="2024-06-01"
    )
elif os.getenv('OPENAI_API_KEY') != "":
    # OpenAIの場合
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
else:
    print("APIKeyの設定を確認してください")

# chatのモデルを取得
model = None
if os.getenv('AZURE_OPENAI_API_KEY') != "":
    # Azureの場合
    model = AzureChatOpenAI(
        azure_deployment="chat",
        openai_api_version="2024-06-01"
    )
elif os.getenv('OPENAI_API_KEY') != "":
    # OpenAIの場合
    model = ChatOpenAI(model="gpt-4")
else:
    print("APIKeyの設定を確認してください")

#### 2.PromptTemplateの設定

In [ ]:
system_prompt = (
    "あなたは質問対応のアシスタントです。"
    "質問に答えるために、検索された文脈の以下の部分を使用してください。"
    "答えがわからない場合は、わからないと答えてください。"
    "回答は3文以内で簡潔にしてください。"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
prompt.invoke({"context": "今日の料理はカレーです", "input": "今日の料理はなんですか？"})

#### 3. FAISSのvector dataを取得

In [ ]:
vectorstore = FAISS.load_local("./db", embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()

#### 4. 類似ドキュメントの取得

In [ ]:
relavant_docs = retriever.invoke("LLMとは何ですか？概要と特徴を教えてください。", k=3)

In [ ]:
# 類似文書を表示
for doc in relavant_docs:
    print(doc)
    print("-----")

#### 5. 解答を得る

In [ ]:
chain = prompt | model
response = chain.invoke({"context": relavant_docs, "input": "LLMとは何ですか？概要と特徴を教えてください。"})

In [ ]:
response.content